This notebook is part of https://github.com/AudioSceneDescriptionFormat/splines, see also https://splines.readthedocs.io/.

# Lagrange Interpolation

Before diving into splines,
let's have a look at an arguably simpler interpolation method using polynomials:
[Lagrange interpolation](https://en.wikipedia.org/wiki/Lagrange_polynomial).

This is easy to implement, but as we will see,
it has quite severe limitations,
which will motivate us to look into splines later.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

Assume we have $N$ time instants $t_i$, with $0 \le i < N$:

In [ ]:
ts = -1.5, 0.5, 1.7, 3, 4

... and for each time instant we are given an associated value $x_i$:

In [ ]:
xs = 2, -1, 1.3, 3.14, 1

Our task is now to find a function that yields
the given $x_i$ values for the given times $t_i$ and
some "reasonable" interpolated values when evaluated at time values in-between.

The idea of Lagrange interpolation is to create a separate polynomial
for each of the $N$ given time instants,
which will be weighted by the associated $x$.
The final interpolation function is the weighted sum of these $N$ polynomials.

In order for this to actually work,
the polynomials must fulfill the following requirements:

* Each polynomial must yield $1$ when evaluated at its associated time $t_i$.
* Each polynomial must yield $0$ at all other instances in the set of given times.

To satisfy the second point,
let's create a product with a term for each of the relevant times
and make each of those factors vanish when evaluated at their associated time.
As an example we look at the basis for $t_3 = 3$:

In [ ]:
def maybe_polynomial_3(t):
    t = np.asarray(t)
    return (t - (-1.5)) * (t - 0.5) * (t - 1.7) * (t - 4)

In [ ]:
maybe_polynomial_3(ts)

As we can see, this indeed fulfills the second requirement.
Note that we were given 5 time instants,
but we need only 4 product terms
(corresponding to the 4 roots of the polynomial).

Now, for the first requirement,
we can divide each term to yield $1$ when evaluated at $t = 3$
(luckily, this will not violate the second requirement).
If each term is $1$, the whole product will also be $1$:

In [ ]:
def polynomial_3(t):
    t = np.asarray(t)
    return (
        (t - (-1.5)) / (3 - (-1.5)) *
        (t - 0.5) / (3 - 0.5) *
        (t - 1.7) / (3 - 1.7) *
        (t - 4) / (3 - 4))

In [ ]:
polynomial_3(ts)

That's it!

To get a better idea what's going on between the given time instances,
let's plot this polynomial
(with a little help from [helper.py](helper.py)):

In [ ]:
from helper import grid_lines

In [ ]:
plot_times = np.linspace(ts[0], ts[-1], 100)

In [ ]:
plt.plot(plot_times, polynomial_3(plot_times))
grid_lines(ts, [0, 1])

We can see from its shape that this is a polynomial of degree 4,
which makes sense because the product we are using has 4 terms
containing one $t$ each.
We can also see that it has the value $0$ at each of the initially provided
time instances $t_i$, except for $t_3 = 3$, where it has the value $1$.

The above calculation can be easily generalized to be able to get
any one of the set of polynomials defined by an arbitrary list of time instants:

In [ ]:
def lagrange_polynomial(times, i, t):
    """i-th Lagrange polynomial for the given time values, evaluated at t."""
    t = np.asarray(t)
    product = np.multiply.reduce
    return product([
        (t - times[j]) / (times[i] - times[j])
        for j in range(len(times))
        if i != j
    ])

Now we can calculate and visualize all 5 polynomials
for our 5 given time instants:

In [ ]:
polys = np.column_stack([lagrange_polynomial(ts, i, plot_times)
                         for i in range(len(ts))])

In [ ]:
plt.plot(plot_times, polys)
grid_lines(ts, [0, 1])

Finally, the interpolated values can be obtained
by applying the given $x_i$ values as weights to the polynomials
and summing everything together:

In [ ]:
weighted_polys = polys * xs

In [ ]:
interpolated = np.sum(weighted_polys, axis=-1)

In [ ]:
plt.plot(plot_times, weighted_polys)
plt.plot(plot_times, interpolated, color='black', linestyle='dashed')
plt.scatter(ts, xs, color='black')
grid_lines(ts)

Lagrange interpolation can of course also be used in higher-dimensional spaces,
for example in two-dimensional space:

In [ ]:
def plot_lagrange_2d(vertices, times, dots_per_second=15, ax=None):
    if ax is None:
        ax = plt.gca()
    total_duration = times[-1] - times[0]
    total_dots = int(total_duration * dots_per_second) + 1
    t = times[0] + np.arange(total_dots) / dots_per_second
    polys = [lagrange_polynomial(times, i, t) for i in range(len(times))]
    weighted_polys = np.expand_dims(np.transpose(vertices), -1) * polys
    interpolated = np.sum(weighted_polys, axis=1)
    ax.plot(
        *np.transpose(vertices),
        linestyle=':', color='lightgrey',
        marker='x', markeredgecolor='black')
    ax.plot(*interpolated, '.')
    ax.axis('equal')

This time, we have a list of two-dimensional vectors
and the same list of associated times as before:

In [ ]:
plot_lagrange_2d([(2, -3), (-1, 0), (1.3, 1), (3.14, 0), (1, -1)], ts)

This seems to work quite well,
but as indicated above,
Lagrange implementation has a severe limitation.
This limitation gets more apparent when using more vertices,
which leads to a higher-degree polynomial.

In [ ]:
vertices = [
    (1, 0),
    (1, 2),
    (3, 0),
    (2, -1),
    (2.5, 1.5),
    (5, 2),
    (6, 1),
    (5, 0),
    (6, -2),
    (7, 2),
    (4, 4),
]
times = range(len(vertices))
plot_lagrange_2d(vertices, times)

Here we see a severe overshooting effect,
most pronounced at the beginning and the end of the curve.
This effect is called
[Runge's phenomenon](https://en.wikipedia.org/wiki/Runge's_phenomenon).

Long story short,
Lagrange interpolation is typically not usable for drawing curves.
For comparison, let's use the same positions and time values
and create a [Catmull--Rom spline](catmull-rom.ipynb):

In [ ]:
import splines

In [ ]:
cr_spline = splines.CatmullRom(vertices, grid=times)

In [ ]:
plt.plot(*cr_spline.evaluate(np.linspace(times[-1], times[0], 150)).T, '.')
plt.plot(
    *np.transpose(vertices),
    linestyle=':', color='lightgrey',
    marker='x', markeredgecolor='black')
plt.axis('equal');

This clearly doesn't have the overshooting problem we saw above.

<div class="alert alert-info">

Note

The [splines.CatmullRom](../python-module/splines.rst#splines.CatmullRom) class
uses ["natural" end conditions](end-conditions-natural.ipynb) by default.

</div>